Adınız: Gökhan
Soyadınız: YAPICI
Numaranız:2112729006
GitHub Repo Bağlantısı:https://github.com/gkhan5252/Yolov8_i-e_Nesne_Tespit_Uygulamasi

In [ ]:
# KÜTÜPHANELERI YÜKLEİ
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pyyaml"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "opencv-python"])

import os, zipfile, shutil, yaml, torch, cv2, random
from pathlib import Path
import matplotlib.pyplot as plt
from ultralytics import YOLO

print("\n" + "="*70)
print("🚀 YOLOV8 COLAB EĞİTİMİ - BAŞLANIYOR")
print("="*70 + "\n")

# 1. ORTAM HAZIRLAMA
print("1️⃣  ORTAM HAZIRLANIYORU...")
print(f"   GPU: {torch.cuda.is_available()}")
print("   ✅ Kütüphaneler yüklendi\n")

# 2. DATASET ÇIKARMA
print("2️⃣  DATASET ÇIKARILIYOR...")
dataset_zip = Path('/content/Dataset1.zip')
if not dataset_zip.exists():
    print("   ⚠️  Dataset1.zip bulunamadı!")
    print("   📁 Files sekmesinden Dataset1.zip yükle ve tekrar çalıştır")
    sys.exit(1)

print(f"   ✅ ZIP dosyası bulundu, çıkarılıyor...")
with zipfile.ZipFile(str(dataset_zip), 'r') as z:
    z.extractall('/content/')

# Ayrı veri setlerini bul (catal, kasik, tabak)
separate_datasets = []
for name in ['catal', 'kasik', 'tabak']:
    dataset_path = Path(f'/content/{name}.v1i.yolov8') or Path(f'/content/{name}')
    if dataset_path.exists():
        train_path = dataset_path / 'train' / 'images'
        if train_path.exists():
            separate_datasets.append((name, dataset_path))
            print(f"   ✅ {name} veri seti bulundu")

if not separate_datasets:
    print("   ⚠️  HATA: catal, kasik, tabak veri setleri bulunamadı!")
    print("   Mevcut klasörler:")
    for item in Path('/content').iterdir():
        if item.is_dir() and not item.name.startswith('.'):
            print(f"     - {item.name}")
    sys.exit(1)

# UNIFIED DATASET OLUŞTUR
print("\n   📦 Birleştirilmiş veri seti oluşturuluyor...")
unified_dir = Path('/content/yolo_dataset')
unified_dir.mkdir(exist_ok=True)

train_images = unified_dir / 'train' / 'images'
train_labels = unified_dir / 'train' / 'labels'
val_images = unified_dir / 'val' / 'images'
val_labels = unified_dir / 'val' / 'labels'

train_images.mkdir(parents=True, exist_ok=True)
train_labels.mkdir(parents=True, exist_ok=True)
val_images.mkdir(parents=True, exist_ok=True)
val_labels.mkdir(parents=True, exist_ok=True)

# Sınıf eşlemesi
class_mapping = {'catal': 0, 'tabak': 1, 'kasik': 2}

total_train = 0
total_val = 0

# Her veri setini işle
all_train_files = []

for class_name, dataset_path in separate_datasets:
    class_id = class_mapping[class_name]

    # Train görüntüleri kopyala
    src_train_imgs = dataset_path / 'train' / 'images'
    if src_train_imgs.exists():
        for img_file in src_train_imgs.glob('*.*'):
            shutil.copy(str(img_file), str(train_images / img_file.name))
            total_train += 1

            # Label dosyasını kopyala ve sınıf ID'sini güncelle
            label_src = dataset_path / 'train' / 'labels' / f'{img_file.stem}.txt'
            label_dst = train_labels / f'{img_file.stem}.txt'
            if label_src.exists():
                with open(label_src, 'r') as f:
                    lines = f.readlines()
                with open(label_dst, 'w') as f:
                    for line in lines:
                        parts = line.strip().split()
                        if parts:
                            parts[0] = str(class_id)
                            f.write(' '.join(parts) + '\n')
                all_train_files.append((img_file.name, f'{img_file.stem}.txt'))

    # Validation görüntüleri kopyala (varsa)
    src_val_imgs = dataset_path / 'val' / 'images'
    if src_val_imgs.exists():
        for img_file in src_val_imgs.glob('*.*'):
            shutil.copy(str(img_file), str(val_images / img_file.name))
            total_val += 1

            label_src = dataset_path / 'val' / 'labels' / f'{img_file.stem}.txt'
            label_dst = val_labels / f'{img_file.stem}.txt'
            if label_src.exists():
                with open(label_src, 'r') as f:
                    lines = f.readlines()
                with open(label_dst, 'w') as f:
                    for line in lines:
                        parts = line.strip().split()
                        if parts:
                            parts[0] = str(class_id)
                            f.write(' '.join(parts) + '\n')

# Eğer validation klasörü boşsa, eğitimden %20 taşı
if total_val == 0 and total_train > 0:
    import random
    random.shuffle(all_train_files)
    split_idx = int(total_train * 0.2)  # %20 validation

    for img_name, label_name in all_train_files[:split_idx]:
        shutil.move(str(train_images / img_name), str(val_images / img_name))
        shutil.move(str(train_labels / label_name), str(val_labels / label_name))
        total_train -= 1
        total_val += 1

print(f"   ✅ {total_train} eğitim + {total_val} doğrulama görüntüsü birleştirildi\n")

dataset_dir = unified_dir

# 3. DATA.YAML HAZIRLAMA
print("3️⃣  CONFIG HAZIRLANIYOR...")
data_yaml = dataset_dir / 'data.yaml'

with open(data_yaml, 'w') as f:
    f.write("""path: /content/yolo_dataset
train: train/images
val: val/images
test: test/images

nc: 3
names:
  0: catal
  1: tabak
  2: kasik
""")

print("   ✅ data.yaml oluşturuldu")

train_imgs = len(list((dataset_dir / 'train' / 'images').glob('*.*')))
val_imgs = len(list((dataset_dir / 'val' / 'images').glob('*.*')))

print(f"   Sınıflar: catal, tabak, kasik")
print(f"   Eğitim: {train_imgs} | Doğrulama: {val_imgs}")
print(f"   ✅ Tamamlandı\n")

# 4. MODEL EĞİTİMİ
print("4️⃣  MODEL EĞİTİMİ BAŞLIYOR (40 epoch)...")
print("   ⏳ Lütfen bekleyin...\n")

device = 0 if torch.cuda.is_available() else 'cpu'
batch = 16 if torch.cuda.is_available() else 8

model = YOLO('yolov8n.pt')
results = model.train(
    data=str(data_yaml),
    epochs=40,
    imgsz=640,
    batch=batch,
    patience=20,
    device=device,
    verbose=False,
    save=True,
    project='/content/runs/detect',
    name='utensil_detection'
)

print("   ✅ Eğitim Tamamlandı\n")

# 5. METRIKLERI GÖSTER
print("5️⃣  MODEL METRİKLERİ...")

best_path = Path('/content/runs/detect/utensil_detection/weights/best.pt')
best_model = YOLO(str(best_path))

val_results = best_model.val(data=str(data_yaml))

print(f"   mAP50:    {val_results.box.map50:.4f}")
print(f"   mAP50-95: {val_results.box.map:.4f}")
print("   ✅ Tamamlandı\n")

# 6. TEST ÖRNEĞİ
print("6️⃣  TEST ÖRNEĞİ...")

train_dir = dataset_dir / 'train' / 'images'
test_imgs = list(train_dir.glob('*.*'))

if test_imgs:
    test_img = random.choice(test_imgs)
    print(f"   Test: {test_img.name}")

    pred = best_model.predict(str(test_img), conf=0.4, verbose=False)

    fig, ax = plt.subplots(figsize=(10, 8))
    result_img = pred[0].plot()
    result_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    ax.imshow(result_rgb)
    ax.set_title('YOLOv8 Tespit Sonucu')
    ax.axis('off')
    plt.tight_layout()
    plt.show()

    count = sum(len(r.boxes) for r in pred)
    print(f"   ✅ {count} nesne tespit edildi\n")

# 7. DOSYA İNDİRME
print("7️⃣  DOSYALARI İNDİRME...")

from google.colab import files

out_model = Path('/content/best_utensil_colab.pt')
shutil.copy(str(best_path), str(out_model))
size = out_model.stat().st_size / (1024**2)

print(f"   📥 best_utensil_colab.pt ({size:.2f} MB)")
files.download(str(out_model))

csv = Path('/content/runs/detect/utensil_detection/results.csv')
if csv.exists():
    print(f"   📥 results.csv")
    files.download(str(csv))

conf = Path('/content/runs/detect/utensil_detection/confusion_matrix.png')
if conf.exists():
    print(f"   📥 confusion_matrix.png")
    files.download(str(conf))

res = Path('/content/runs/detect/utensil_detection/results.png')
if res.exists():
    print(f"   📥 results.png")
    files.download(str(res))

print("   ✅ Tamamlandı\n")

# 8. SELESAI
print("="*70)
print("✅ TÜM İŞLEMLER TAMAMLANDI!")
print("="*70)
print("\n📌 ÖNEMLİ:")
print("   1. best_utensil_colab.pt dosyasını kaydet")
print("   2. Proje klasörüne kopyala (d:\\model egitim\\)")
print("   3. gui_app.py'de model adını güncelle:")
print("      model = YOLO('best_utensil_colab.pt')")
print("   4. python gui_app.py çalıştır\n")
print("🎉 Başarı!\n")